In [ ]:
# running on Runpod
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir("/content/drive/MyDrive/rag-benchmarks")

In [ ]:
import os
os.chdir('..')
print(os.getcwd())
print(os.listdir())

In [ ]:
import torch

def get_best_device():
    if torch.cuda.is_available():
        print("Found CUDA GPU.")
    elif torch.backends.mps.is_available():
        print("Found Apple Silicon GPU.")
    else:
        print("No GPU found. Using 'cpu'.")

print(os.getcwd())
get_best_device()


In [ ]:
# fetched from run_all_combinations.py with scope limited to scidocs dataset
import subprocess
import sys
import json
import pandas as pd
import os

# Define retrieval and rerank combos
COMBINATIONS = [
    # Sparse retrieval
    {"retrieval": "tfidf", "rerank": None, "desc": "TFIDF + no reranking (BASELINE)"},
    {"retrieval": "tfidf", "rerank": "bi_encoder", "desc": "TFIDF + bi encoder rerank"},
    {"retrieval": "tfidf", "rerank": "cross_encoder", "desc": "TFIDF + cross encoder rerank"},
    {"retrieval": "bm25", "rerank": None, "desc": "BM25 + No reranking"},
    {"retrieval": "bm25", "rerank": "bi_encoder", "desc": "BM25 + bi encoder rerank"},
    {"retrieval": "bm25", "rerank": "cross_encoder", "desc": "BM25 + cross encoder rerank"},
    # Dense retrieval
    {"retrieval": "dense", "rerank": None, "desc": "Dense + no rerank"},
    {"retrieval": "dense", "rerank": "bi_encoder", "desc": "Dense + bi encoder"},
    {"retrieval": "dense", "rerank": "cross_encoder", "desc": "Dense + cross encoder"},
    # Hybrid retrieval (RRF)
    {"retrieval": "hybrid", "rerank": None, "desc": "Hybrid + no rerank"},
    {"retrieval": "hybrid", "rerank": "bi_encoder", "desc": "Hybrid + bi encoder"},
    {"retrieval": "hybrid", "rerank": "cross_encoder", "desc": "Hybrid + cross encoder"},
]

dataset_to_run = "scidocs"
output_dir = 'outputs'
os.makedirs(output_dir, exist_ok=True)
all_results = []

print(f"=== Running experiments for dataset: {dataset_to_run} ===")
for combo in COMBINATIONS:
    print(f"--- {combo['desc']} ---")
    try:
        reranker_name = combo["rerank"].split('_')[0] if combo["rerank"] else "none"
        output_filename = f"{dataset_to_run}_{combo['retrieval']}_{reranker_name}.json"
        output_path = os.path.join(output_dir, output_filename)

        command = [
            "python",
            "scripts/run_experiment.py",
            "--dataset",
            dataset_to_run,
            "--retriever",
            combo["retrieval"],
            "--reranker",
            reranker_name,
            "--output",
            output_path,
            "--top_k",
            "100",
            "--device",
            "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu",
        ]

        # Use ! to run the command in the notebook
        # This is equivalent to subprocess.run but shows output live
        !{' '.join(command)}

        with open(output_path, 'r') as f:
            results = json.load(f)

        results['dataset'] = dataset_to_run
        results['retriever'] = combo['retrieval']
        results['reranker'] = combo['rerank'] if combo['rerank'] else "none"
        all_results.append(results)

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if all_results:
    results_df = pd.DataFrame(all_results)
    results_csv_path = f'{dataset_to_run}_results.csv'
    results_df.to_csv(results_csv_path, index=False)
    print(f"All experiments complete. Results saved to {results_csv_path}")
    display(results_df)
